In [1]:
import pandas as pd
import numpy as np
import copy
import os
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import crawling_boat as cb
import math
import optuna
import gc

import config.config as cfg
import util.modules as mdl

LOAD_DATA_COLUMNS = ['race_date', 'place_id', 'race_no', 'race_date_no', 'bracket_no', 'is_miss', 'player_id', 'player_grade', 'branch', 'born_area', 'age', 'weight', 'f_count', 'l_count', 'start_time_avg', 'first_rate_all', 'second_rate_all', 'third_rate_all', 'first_rate_area', 'second_rate_area', 'third_rate_area', 'motor_no', 'motor_within_second_rate', 'motor_within_third_rate', 'boat_no', 'boat_within_second_rate', 'boat_within_third_rate', 'pre_time', 'tilt_angle', 'propeller', 'parts', 'adjust_weight', 'pre_start_timing', 'finish_order', 'player_race_time', 'start_timing', 'win_pattern', 'race_grade', 'distance', 'course_direction', 'weather', 'temperature', 'wind', 'wind_direction', 'water_temperature', 'wave_height', 'race_date_old1', 'place_id_old1', 'race_no_old1', 'bracket_no_old1', 'is_miss_old1', 'player_id_old1', 'player_grade_old1', 'branch_old1', 'born_area_old1', 'age_old1', 'weight_old1', 'f_count_old1', 'l_count_old1', 'start_time_avg_old1', 'first_rate_all_old1', 'second_rate_all_old1', 'third_rate_all_old1', 'first_rate_area_old1', 'second_rate_area_old1', 'third_rate_area_old1', 'motor_no_old1', 'motor_within_second_rate_old1', 'motor_within_third_rate_old1', 'boat_no_old1', 'boat_within_second_rate_old1', 'boat_within_third_rate_old1', 'pre_time_old1', 'tilt_angle_old1', 'propeller_old1', 'parts_old1', 'adjust_weight_old1', 'pre_start_timing_old1', 'finish_order_old1', 'player_race_time_old1', 'start_timing_old1', 'win_pattern_old1', 'race_date_old2', 'place_id_old2', 'race_no_old2', 'bracket_no_old2', 'is_miss_old2', 'player_id_old2', 'player_grade_old2', 'branch_old2', 'born_area_old2', 'age_old2', 'weight_old2', 'f_count_old2', 'l_count_old2', 'start_time_avg_old2', 'first_rate_all_old2', 'second_rate_all_old2', 'third_rate_all_old2', 'first_rate_area_old2', 'second_rate_area_old2', 'third_rate_area_old2', 'motor_no_old2', 'motor_within_second_rate_old2', 'motor_within_third_rate_old2', 'boat_no_old2', 'boat_within_second_rate_old2', 'boat_within_third_rate_old2', 'pre_time_old2', 'tilt_angle_old2', 'propeller_old2', 'parts_old2', 'adjust_weight_old2', 'pre_start_timing_old2', 'finish_order_old2', 'player_race_time_old2', 'start_timing_old2', 'win_pattern_old2', 'race_date_old3', 'place_id_old3', 'race_no_old3', 'bracket_no_old3', 'is_miss_old3', 'player_id_old3', 'player_grade_old3', 'branch_old3', 'born_area_old3', 'age_old3', 'weight_old3', 'f_count_old3', 'l_count_old3', 'start_time_avg_old3', 'first_rate_all_old3', 'second_rate_all_old3', 'third_rate_all_old3', 'first_rate_area_old3', 'second_rate_area_old3', 'third_rate_area_old3', 'motor_no_old3', 'motor_within_second_rate_old3', 'motor_within_third_rate_old3', 'boat_no_old3', 'boat_within_second_rate_old3', 'boat_within_third_rate_old3', 'pre_time_old3', 'tilt_angle_old3', 'propeller_old3', 'parts_old3', 'adjust_weight_old3', 'pre_start_timing_old3', 'finish_order_old3', 'player_race_time_old3', 'start_timing_old3', 'win_pattern_old3', 'race_grade_old1',
                     'distance_old1', 'course_direction_old1', 'weather_old1', 'temperature_old1', 'wind_old1', 'wind_direction_old1', 'water_temperature_old1', 'wave_height_old1', 
                     'race_grade_old2', 'distance_old2', 'course_direction_old2', 'weather_old2', 'temperature_old2', 'wind_old2', 'wind_direction_old2', 'water_temperature_old2', 'wave_height_old2', 
                     'race_grade_old3', 'distance_old3', 'course_direction_old3', 'weather_old3', 'temperature_old3', 'wind_old3', 'wind_direction_old3', 'water_temperature_old3', 'wave_height_old3']

In [2]:
data = pd.read_csv('csv/boat_race_data_tmp.csv')
dtypes = data.dtypes

/Users/yuta_miyawaki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,50,86,122) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
dtypes = pd.read_csv('csv/dtypes.csv', header = -1, index_col=0)[1]

In [25]:
dtypes[1]

0
race_date                     int64
place_id                      int64
race_no                       int64
race_date_no                float64
bracket_no                    int64
is_miss                      object
player_id                   float64
player_grade                 object
branch                       object
born_area                    object
age                           int64
weight                        int64
f_count                       int64
l_count                       int64
start_time_avg              float64
first_rate_all              float64
second_rate_all             float64
third_rate_all              float64
first_rate_area             float64
second_rate_area            float64
third_rate_area             float64
motor_no                      int64
motor_within_second_rate    float64
motor_within_third_rate     float64
boat_no                       int64
boat_within_second_rate     float64
boat_within_third_rate      float64
pre_time                  

In [11]:
pd.DataFrame(dtypes)[0]

race_date                     int64
place_id                      int64
race_no                       int64
race_date_no                float64
bracket_no                    int64
is_miss                      object
player_id                   float64
player_grade                 object
branch                       object
born_area                    object
age                           int64
weight                        int64
f_count                       int64
l_count                       int64
start_time_avg              float64
first_rate_all              float64
second_rate_all             float64
third_rate_all              float64
first_rate_area             float64
second_rate_area            float64
third_rate_area             float64
motor_no                      int64
motor_within_second_rate    float64
motor_within_third_rate     float64
boat_no                       int64
boat_within_second_rate     float64
boat_within_third_rate      float64
pre_time                    

In [148]:
PARTS = {'キャリボ': 4,
                 'ピストン': 4,
                 'リング': 3,
                 '電気': 2, 
                 'キャブ': 3,
                 'ギヤ': 2,
                 'シリンダ': 4,
                 'シャフト': 4}
def parts_count(x, key, num):
    x = str(x)
    if x.find(key) != -1:
        if x[x.find(key):].find('×') != -1:
            return x[x.find(key)+num+1]
        else:
            return 1
    else:
        return 0

def drop_miss(df):
    return df[df['is_miss'] == False]

def conv_time(x):
    if str(x).find('nan') != -1:
        return x
    elif str(x).find(':') == -1:
        return np.nan
    else:
        time = x.split(':')
        return float(time[0])*60 + float(time[1]) + float(time[2])/10
        
def preprocessing(df):
    # 月
    df['month'] = df['race_date'].astype(int).apply(lambda x: (x%10000)//100)
    df['month_old1'] = df['race_date_old1'].astype(int).apply(lambda x: (x%10000)//100)
    df['month_old2'] = df['race_date_old2'].astype(int).apply(lambda x: (x%10000)//100)
    df['month_old3'] = df['race_date_old3'].astype(int).apply(lambda x: (x%10000)//100)
    # 日
    df['date'] = df['race_date'].astype(int).apply(lambda x: x%100)
    df['date_old1'] = df['race_date_old1'].astype(int).apply(lambda x: x%100)
    df['date_old2'] = df['race_date_old2'].astype(int).apply(lambda x: x%100)
    df['date_old3'] = df['race_date_old3'].astype(int).apply(lambda x: x%100)
    
    # 周期データを三角関数に変換
    # 月
    df['month'] = df['month'].astype(int)
    df['month_old1'] = df['month_old1'].astype(int)
    df['month_old2'] = df['month_old2'].astype(int)
    df['month_old3'] = df['month_old3'].astype(int)
    
    df['date'] = df['date'].astype(int)
    df['date_old1'] = df['date_old1'].astype(int)
    df['date_old2'] = df['date_old2'].astype(int)
    df['date_old3'] = df['date_old3'].astype(int)
    
    df['course_direction'] = df['course_direction'].astype(int)
    df['course_direction_old1'] = df['course_direction_old1'].astype(int)
    df['course_direction_old2'] = df['course_direction_old2'].astype(int)
    df['course_direction_old3'] = df['course_direction_old3'].astype(int)
    
    df['wind_direction'] = df['wind_direction'].astype(int)
    df['wind_direction_old1'] = df['wind_direction_old1'].astype(int)
    df['wind_direction_old2'] = df['wind_direction_old2'].astype(int)
    df['wind_direction_old3'] = df['wind_direction_old3'].astype(int)
    
    
    df['month_cos'] = np.cos(2 * np.pi * df['month']/df['month'].max())
    df['month_sin'] = np.sin(2 * np.pi * df['month']/df['month'].max())
    df['month_cos_old1'] = np.cos(2 * np.pi * df['month_old1']/df['month_old1'].max())
    df['month_sin_old1'] = np.sin(2 * np.pi * df['month_old1']/df['month_old1'].max())
    df['month_cos_old2'] = np.cos(2 * np.pi * df['month_old2']/df['month_old2'].max())
    df['month_sin_old2'] = np.sin(2 * np.pi * df['month_old2']/df['month_old2'].max())
    df['month_cos_old3'] = np.cos(2 * np.pi * df['month_old3']/df['month_old3'].max())
    df['month_sin_old3'] = np.sin(2 * np.pi * df['month_old3']/df['month_old3'].max())
    # 日
    df['date_cos'] = np.cos(2 * np.pi * df['date']/df['date'].max())
    df['date_sin'] = np.sin(2 * np.pi * df['date']/df['date'].max())
    df['date_cos_old1'] = np.cos(2 * np.pi * df['date_old1']/df['date_old1'].max())
    df['date_sin_old1'] = np.sin(2 * np.pi * df['date_old1']/df['date_old1'].max())
    df['date_cos_old2'] = np.cos(2 * np.pi * df['date_old2']/df['date_old2'].max())
    df['date_sin_old2'] = np.sin(2 * np.pi * df['date_old2']/df['date_old2'].max())
    df['date_cos_old3'] = np.cos(2 * np.pi * df['date_old3']/df['date_old3'].max())
    df['date_sin_old3'] = np.sin(2 * np.pi * df['date_old3']/df['date_old3'].max())
    # コース方向
    df['course_direction_cos'] = np.cos(2 * np.pi * df['course_direction']/df['course_direction'].max())
    df['course_direction_sin'] = np.sin(2 * np.pi * df['course_direction']/df['course_direction'].max())
    df['course_direction_cos_old1'] = np.cos(2 * np.pi * df['course_direction_old1']/df['course_direction_old1'].max())
    df['course_direction_sin_old1'] = np.sin(2 * np.pi * df['course_direction_old1']/df['course_direction_old1'].max())
    df['course_direction_cos_old2'] = np.cos(2 * np.pi * df['course_direction_old2']/df['course_direction_old2'].max())
    df['course_direction_sin_old2'] = np.sin(2 * np.pi * df['course_direction_old2']/df['course_direction_old2'].max())
    df['course_direction_cos_old3'] = np.cos(2 * np.pi * df['course_direction_old3']/df['course_direction_old3'].max())
    df['course_direction_sin_old3'] = np.sin(2 * np.pi * df['course_direction_old3']/df['course_direction_old3'].max())
    # 風方向
    df['wind_direction_cos'] = np.cos(2 * np.pi * df['wind_direction']/df['wind_direction'].max())
    df['wind_direction_sin'] = np.sin(2 * np.pi * df['wind_direction']/df['wind_direction'].max())
    df['wind_direction_cos_old1'] = np.cos(2 * np.pi * df['wind_direction_old1']/df['wind_direction_old1'].max())
    df['wind_direction_sin_old1'] = np.sin(2 * np.pi * df['wind_direction_old1']/df['wind_direction_old1'].max())
    df['wind_direction_cos_old2'] = np.cos(2 * np.pi * df['wind_direction_old2']/df['wind_direction_old2'].max())
    df['wind_direction_sin_old2'] = np.sin(2 * np.pi * df['wind_direction_old2']/df['wind_direction_old2'].max())
    df['wind_direction_cos_old3'] = np.cos(2 * np.pi * df['wind_direction_old3']/df['wind_direction_old3'].max())
    df['wind_direction_sin_old3'] = np.sin(2 * np.pi * df['wind_direction_old3']/df['wind_direction_old3'].max())
    # 風とコースの差分方向
    df['course_wind_direction_cos'] = np.cos(2 * np.pi * (df['wind_direction']/df['wind_direction'].max() - df['course_direction']/df['course_direction'].max()))
    df['course_wind_direction_sin'] = np.sin(2 * np.pi * (df['wind_direction']/df['wind_direction'].max() - df['course_direction']/df['course_direction'].max()))
    df['course_wind_direction_cos_old1'] = np.cos(2 * np.pi * (df['wind_direction_old1']/df['wind_direction_old1'].max() - df['course_direction_old1']/df['course_direction_old1'].max()))
    df['course_wind_direction_sin_old1'] = np.sin(2 * np.pi * (df['wind_direction_old1']/df['wind_direction_old1'].max() - df['course_direction_old1']/df['course_direction_old1'].max()))
    df['course_wind_direction_cos_old2'] = np.cos(2 * np.pi * (df['wind_direction_old2']/df['wind_direction_old2'].max() - df['course_direction_old2']/df['course_direction_old2'].max()))
    df['course_wind_direction_sin_old2'] = np.sin(2 * np.pi * (df['wind_direction_old2']/df['wind_direction_old2'].max() - df['course_direction_old2']/df['course_direction_old2'].max()))
    df['course_wind_direction_cos_old3'] = np.cos(2 * np.pi * (df['wind_direction_old3']/df['wind_direction_old3'].max() - df['course_direction_old3']/df['course_direction_old3'].max()))
    df['course_wind_direction_sin_old3'] = np.sin(2 * np.pi * (df['wind_direction_old3']/df['wind_direction_old3'].max() - df['course_direction_old3']/df['course_direction_old3'].max()))
    
    # 展示フライングフラグ
    df['pre_f_flg'] = df['pre_start_timing'].apply(lambda x: str(x).find('F')!=-1)
    df['pre_l_flg'] = df['pre_start_timing'].apply(lambda x: str(x).find('L')!=-1)
    df['pre_start_timing'] = df['pre_start_timing'].apply(lambda x: -1*float(x.replace('F', '0')) if str(x).find('F')!=-1 else (np.nan if str(x).find('L')!=-1 else float(x)))
    df['pre_f_flg_old1'] = df['pre_start_timing_old1'].apply(lambda x: str(x).find('F')!=-1)
    df['pre_l_flg_old1'] = df['pre_start_timing_old1'].apply(lambda x: str(x).find('L')!=-1)
    df['pre_start_timing_old1'] = df['pre_start_timing_old1'].apply(lambda x: -1*float(x.replace('F', '0')) if str(x).find('F')!=-1 else (np.nan if str(x).find('L')!=-1 else float(x)))
    df['start_timing_old1'] = df['start_timing_old1'].apply(lambda x: -1*float(x.replace('F', '0')) if str(x).find('F')!=-1 else (np.nan if str(x).find('L')!=-1 else float(x)))
    df['pre_f_flg_old2'] = df['pre_start_timing_old2'].apply(lambda x: str(x).find('F')!=-1)
    df['pre_l_flg_old2'] = df['pre_start_timing_old2'].apply(lambda x: str(x).find('L')!=-1)
    df['pre_start_timing_old2'] = df['pre_start_timing_old2'].apply(lambda x: -1*float(x.replace('F', '0')) if str(x).find('F')!=-1 else (np.nan if str(x).find('L')!=-1 else float(x)))
    df['start_timing_old2'] = df['start_timing_old2'].apply(lambda x: -1*float(x.replace('F', '0')) if str(x).find('F')!=-1 else (np.nan if str(x).find('L')!=-1 else float(x)))
    df['pre_f_flg_old3'] = df['pre_start_timing_old3'].apply(lambda x: str(x).find('F')!=-1)
    df['pre_l_flg_old3'] = df['pre_start_timing_old3'].apply(lambda x: str(x).find('L')!=-1)
    df['pre_start_timing_old3'] = df['pre_start_timing_old3'].apply(lambda x: -1*float(x.replace('F', '0')) if str(x).find('F')!=-1 else (np.nan if str(x).find('L')!=-1 else float(x)))
    df['start_timing_old3'] = df['start_timing_old3'].apply(lambda x: -1*float(x.replace('F', '0')) if str(x).find('F')!=-1 else (np.nan if str(x).find('L')!=-1 else float(x)))
    
    # パーツフラグと個数
    for key, values in PARTS.items():
        df[key+'_flg'] = df['parts'].apply(lambda x: True if str(x).find(key)!=-1 else False)
        df[key+'_count'] = df['parts'].apply(parts_count, key=key, num=PARTS[key])
        df[key+'_flg_old1'] = df['parts_old1'].apply(lambda x: True if str(x).find(key)!=-1 else False)
        df[key+'_count_old1'] = df['parts_old1'].apply(parts_count, key=key, num=PARTS[key])
        df[key+'_flg_old2'] = df['parts_old2'].apply(lambda x: True if str(x).find(key)!=-1 else False)
        df[key+'_count_old2'] = df['parts_old2'].apply(parts_count, key=key, num=PARTS[key])
        df[key+'_flg_old3'] = df['parts_old3'].apply(lambda x: True if str(x).find(key)!=-1 else False)
        df[key+'_count_old3'] = df['parts_old3'].apply(parts_count, key=key, num=PARTS[key])
        
    # 枠番
    df['bracket_num'] = df['bracket_no']
    df['bracket_num_old1'] = df['bracket_no_old1']
    df['bracket_num_old2'] = df['bracket_no_old2']
    df['bracket_num_old3'] = df['bracket_no_old3']
    
    # 過去レースタイム変換
    df['player_race_time_old1'] = df['player_race_time_old1'].apply(conv_time)
    df['player_race_time_old2'] = df['player_race_time_old2'].apply(conv_time)
    df['player_race_time_old3'] = df['player_race_time_old3'].apply(conv_time)
    
    # 特選フラグ
    # TODO
    
    # 前走からの日付
    #コンドツクル
    
    # ターゲットエンコーディング
    group = df.groupby(['bracket_no'])
    # 枠順ごとの着順平均
    bracket_order_mean = pd.read_csv('bracket_order_mean.csv')
    bracket_order_mean['bracket_no'] = bracket_order_mean.index+1
    df = pd.merge(df, bracket_order_mean, on='bracket_no')
    # 枠順ごとの着順中央値
    bracket_order_median = pd.read_csv('bracket_order_median.csv')
    bracket_order_median['bracket_no'] = bracket_order_median.index+1
    df = pd.merge(df, bracket_order_median, on='bracket_no')
    
    bracket_order_mean.to_csv('bracket_order_mean.csv', index=False)
    bracket_order_median.to_csv('bracket_order_median.csv', index=False)
    
    df = df.sort_values(by=['race_date', 'place_id', 'race_no', 'bracket_no'])
    
    # n走前nullフラグ作成
    df['old1_is_null'] = df['race_date_old1'].apply(lambda x: np.isnan(x))
    df['old2_is_null'] = df['race_date_old2'].apply(lambda x: np.isnan(x))
    df['old3_is_null'] = df['race_date_old3'].apply(lambda x: np.isnan(x))
    
    # レース内統計
    group = df.groupby(['race_date', 'place_id', 'race_no'])
    stat_col = [
            'age', 'weight', 'f_count',
            'l_count', 'start_time_avg', 'first_rate_all', 'second_rate_all',
            'third_rate_all', 'first_rate_area', 'second_rate_area',
            'third_rate_area', 'motor_within_second_rate',
            'motor_within_third_rate', 'boat_within_second_rate',
            'boat_within_third_rate', 'pre_time', 'tilt_angle', 'adjust_weight'
            ]
    name_mean = {}
    name_std = {}
    name_max = {}
    name_min = {}
    name_rank = {}
    for c in stat_col:
        name_mean[c] = c + '_mean'
        name_std[c] = c + '_std'
        name_max[c] = c + '_max'
        name_min[c] = c + '_min'
    
    mean_data = group[stat_col].mean()
    std_data = group[stat_col].std()
    max_data = group[stat_col].max()
    min_data = group[stat_col].min()
    
    mean_data = mean_data.rename(columns = name_mean)
    std_data = std_data.rename(columns = name_std)
    max_data = max_data.rename(columns = name_max)
    min_data = min_data.rename(columns = name_min)
    
    df = pd.merge(df, mean_data, on = ['race_date', 'place_id','race_no'])
    df = pd.merge(df, std_data, on = ['race_date', 'place_id','race_no'])
    df = pd.merge(df, max_data, on = ['race_date', 'place_id','race_no'])
    df = pd.merge(df, min_data, on = ['race_date', 'place_id','race_no'])
    
    # ランキング
    group = df.groupby(['race_date', 'place_id', 'race_no'])
    rank_col = [
            'age', 'weight', 'start_time_avg', 'first_rate_all', 'second_rate_all',
            'third_rate_all', 'first_rate_area', 'second_rate_area',
            'third_rate_area', 'motor_within_second_rate',
            'motor_within_third_rate', 'boat_within_second_rate',
            'boat_within_third_rate', 'pre_time'
            ]
    for c in stat_col:
        name_rank[c] = c + '_rank'

    rank_data = group[rank_col].rank(ascending=True,method="first")
    rank_data = rank_data.rename(columns = name_rank)
    df = pd.concat([df, rank_data], axis = 1)
    
    # レース情報
    race_list = df[['race_date', 'place_id', 'race_no', 'bracket_no']]
    
    #One-Hot Encoding
    dummies_list = ['place_id','bracket_no','player_grade','branch','born_area','propeller','weather']
    past_dummies_list = ['place_id','bracket_no','player_grade','branch','born_area','propeller','weather','win_pattern']
    
    for c in past_dummies_list:
        for i in range(3):
            dummies_list.append(c+'_old'+str(i+1))
    for i in range(3):
        dummies_list.append('old'+str(i+1)+'_is_null')
    for key in PARTS.keys():
        dummies_list.append(key+'_flg')
        
    df = pd.get_dummies(data = df, columns = dummies_list)
    
    # 目的変数の抽出
    target = {}
    target['finish_order'] = df['finish_order']
    target['player_race_time'] = df['player_race_time']
    
    # 不要カラムの削除
    drop_list = ['race_date','race_date_no','is_miss','player_id','parts','parts_old1','parts_old2','parts_old3',
                        'motor_no','boat_no','finish_order','player_race_time','race_grade',
                        'start_timing','win_pattern','race_date_old1',
                        'is_miss_old1','player_id_old1','motor_no_old1',
                        'boat_no_old1','race_date_old2','is_miss_old2',
                        'player_id_old2','motor_no_old2','boat_no_old2',
                        'race_date_old3','is_miss_old3','player_id_old3',
                        'motor_no_old3','boat_no_old3',
                        'race_grade_old1', 'race_grade_old2', 'race_grade_old3']
    df.drop(drop_list, axis = 1, inplace=True)
    
    input_shape = pd.read_csv('input_shape.csv')
    add_columns = list(filter(lambda x: x not in df.columns, input_shape.columns))
    
    df = pd.concat([df,input_shape])[input_shape.columns]
    
    for c in add_columns:
        df[c] = df[c].fillna(0)
    
    return [df, target, race_list]

In [149]:
def get_predict_data(race_date, place_id, race_no):
    # データ取得クエリ作成
    with open(os.path.join('sql/', 'select_predict_data.sql')) as f:
        query = f.read()

    race_inf = {
                'race_date': race_date,
                'place_id': place_id,
                'race_no': race_no
               }
    
    query = query.format(**race_inf).replace('\n', ' ')
    
    #クエリ実行
    engine = mdl.load_engine()
    res = engine.execute(query)
    
    return [r for r in res]


In [177]:
df = pd.DataFrame(get_predict_data('20200113', '16', '1'), columns=LOAD_DATA_COLUMNS)

In [178]:
df['weight'] = df['weight'].apply(lambda x: math.floor(float(x)))
df['weight_old1'] = df['weight_old1'].apply(lambda x: math.floor(float(x)))
df['weight_old2'] = df['weight_old2'].apply(lambda x: math.floor(float(x)))
df['weight_old3'] = df['weight_old3'].apply(lambda x: math.floor(float(x)))

In [179]:
for i, c in enumerate(LOAD_DATA_COLUMNS):
    df[c] = df[c].astype(dtypes[i])

In [180]:
df, target, race_list = preprocessing(df)

/Users/yuta_miyawaki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:266: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [181]:
df

,race_no,age,weight,f_count,l_count,start_time_avg,first_rate_all,second_rate_all,third_rate_all,first_rate_area,...,電気_flg_False,電気_flg_True,キャブ_flg_False,キャブ_flg_True,ギヤ_flg_False,ギヤ_flg_True,シリンダ_flg_False,シリンダ_flg_True,シャフト_flg_False,シャフト_flg_True
0,1,30,52,0,0,0.18,5.56,37.19,57.02,5.22,...,1,0,1,0,1,0,1,0,1,0
1,1,32,50,0,0,0.17,4.94,25.35,50.70,0.00,...,1,0,1,0,1,0,1,0,1,0
2,1,44,52,0,0,0.19,4.30,22.22,40.00,4.89,...,1,0,1,0,1,0,1,0,1,0
3,1,49,55,0,0,0.19,4.24,23.53,32.35,3.09,...,1,0,1,0,1,0,1,0,1,0
4,1,21,55,0,0,0.22,1.64,4.44,4.44,2.89,...,1,0,1,0,1,0,1,0,1,0
5,1,40,51,0,0,0.19,3.79,11.90,27.38,3.33,...,1,0,1,0,1,0,1,0,1,0


In [182]:
model = lgb.Booster(model_file='model/first_model_target.txt')

In [183]:
output = model.predict(df.values)

In [184]:
output[:,1]/sum(output[:,1])

array([0.60140782, 0.2195434 , 0.09868675, 0.05826815, 0.0075651 ,
       0.01452878])

In [175]:
data

,race_date,place_id,race_no,race_date_no,bracket_no,is_miss,player_id,player_grade,branch,born_area,...,wave_height_old2,race_grade_old3,distance_old3,course_direction_old3,weather_old3,temperature_old3,wind_old3,wind_direction_old3,water_temperature_old3,wave_height_old3
0,20160314,1,1,2.016031e+09,1,False,3883.0,B1,広島,広島,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
1,20160314,1,1,2.016031e+09,2,False,2787.0,B1,群馬,群馬,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
2,20160314,1,1,2.016031e+09,3,False,4167.0,B1,徳島,徳島,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
3,20160314,1,1,2.016031e+09,4,False,4089.0,A1,岡山,岡山,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
4,20160314,1,1,2.016031e+09,5,False,3626.0,B1,大阪,大阪,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
5,20160314,1,1,2.016031e+09,6,False,4807.0,B2,東京,神奈川,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
6,20160314,1,10,2.016031e+09,1,False,3870.0,A2,福井,福井,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
7,20160314,1,10,2.016031e+09,2,False,4089.0,A1,岡山,岡山,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
8,20160314,1,10,2.016031e+09,3,False,3419.0,B1,群馬,群馬,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0
9,20160314,1,10,2.016031e+09,4,False,3478.0,B1,香川,香川,...,0.0,NaN,0,NaN,NaN,0.0,0,NaN,0.0,0.0


In [176]:
[c for c in data.columns]

['race_date',
 'place_id',
 'race_no',
 'race_date_no',
 'bracket_no',
 'is_miss',
 'player_id',
 'player_grade',
 'branch',
 'born_area',
 'age',
 'weight',
 'f_count',
 'l_count',
 'start_time_avg',
 'first_rate_all',
 'second_rate_all',
 'third_rate_all',
 'first_rate_area',
 'second_rate_area',
 'third_rate_area',
 'motor_no',
 'motor_within_second_rate',
 'motor_within_third_rate',
 'boat_no',
 'boat_within_second_rate',
 'boat_within_third_rate',
 'pre_time',
 'tilt_angle',
 'propeller',
 'parts',
 'adjust_weight',
 'pre_start_timing',
 'finish_order',
 'player_race_time',
 'start_timing',
 'win_pattern',
 'race_grade',
 'distance',
 'course_direction',
 'weather',
 'temperature',
 'wind',
 'wind_direction',
 'water_temperature',
 'wave_height',
 'race_date_old1',
 'place_id_old1',
 'race_no_old1',
 'bracket_no_old1',
 'is_miss_old1',
 'player_id_old1',
 'player_grade_old1',
 'branch_old1',
 'born_area_old1',
 'age_old1',
 'weight_old1',
 'f_count_old1',
 'l_count_old1',
 'start_